# MACHINE TRANSLATION WITH FAIRSEQ (1/2) - ENGLISH_TO_TAMIL

Facebook's AI team [recently released](https://github.com/pytorch/fairseq/tree/master/examples/wmt20) its machine translation models for Tamil<->English, and I was eager to give it a try since Tamil is among the most under-served languages in machine learning, and related language pairs are pretty hard to come by.

This is also a good opportunity to try out FB's Fairseq library, which seems pretty user-friendly for folks who work with pandas dataframes. Over two short notebooks, I'll demo a simple workflow for:
* translating English speeches and news stories to Tamil
* translating Tamil speeches and news stories to English.

## DATASET
The datasets used in these two notebooks involve some existing ones already in use by other notebooks in this repo, as well as new ones selected for this exercise.

* Existing datasets:
    - The English speeches are taken from an existing dataset comprising 11 speeches in 4 languages (English, Malay, Chinese, and Tamil) from the website of the [Singapore Prime Minister's Office](https://www.pmo.gov.sg/).
    - The English news stories (on Covid-19) were published on Singapore news outlet's CNA's website in March 2020, and were used in earlier notebooks on English-to-Chinese translation.

* New datasets:
    - The Tamil speeches ([tamil_speeches.csv](https://github.com/chuachinhon/practical_nlp/blob/master/data/tamil_speeches.csv)) are also taken from the website of the Singapore Prime Minister's Office. These are official speeches with English versions (not direct translations though) for reference.
    - The Tamil news stories ([tamil_stories.csv](https://github.com/chuachinhon/practical_nlp/blob/master/data/tamil_stories.csv)) are taken from [Sethi](https://seithi.mediacorp.sg/), Singapore state broadcaster Mediacorp's Tamil-language news website

I kept the new datasets small so as to keep the run times manageable.

## RESULTS
The output CSV files with the translated text and original copy can be [downloaded here](https://www.dropbox.com/sh/26b7q5tqyqp8kim/AAB6HGPhmrF4_Ijdr0NN8hAFa). 

There are still some obvious problems with the quality of the translation. But as mentioned in earlier notebooks, machine translation gets about 70-80% of the job done, allowing human translators to work more efficiently.

I've not done comparisons to see whether the machine translation results from fairseq are better/comparable with those from MarianMT.

In [1]:
import pandas as pd
import re
import torch
import warnings

from nltk.tokenize import sent_tokenize
warnings.filterwarnings('ignore')

# 1. TRANSLATE ENGLISH SPEECHES TO TAMIL

In this notebook, we'll try out Faireseq and FB's English-to-Tamil language pairing. Most of the complex code have been abstracted away by the fairseq API, which is very easy to use with pandas and nltk's sentence tokenizer.

Max token length for the Fairseq models are 1,024. The speeches and stories used here are well under this limit (and to re-iterate, I've used a sentence tokenizer). Longer corpuses will require a separate function to split up the text, which I won't go into here.

In [2]:
# this toy dataset is in the repo. We'll pick only the English speeches

raw = pd.read_csv('../data/translation_speeches.csv')

speeches = raw[raw['Language'] == 'English'].copy()

In [3]:
speeches.head()

,Date,Speaker,Title,Language,Text,URL
0,2020-04-30,Lee Hsien Loong,May Day Message 2020,English,"This year, we mark May Day amidst difficult ci...",https://www.pmo.gov.sg/Newsroom/PM-Lee-Hsien-L...
4,2020-07-27,Lee Hsien Loong,Cabinet Swearing-in Ceremony,English,Singaporeans have just gone through a crucial ...,https://www.pmo.gov.sg/Newsroom/Speech-by-PM-L...
7,2020-08-09,Lee Hsien Loong,National Day Message 2020,English,"Every year, rain or shine, Singaporeans come t...",https://www.pmo.gov.sg/Newsroom/National-Day-M...


## 1.1 CLEAN TEXT

For machine translation, my preference is to clean as lightly as possible so as to preserve as much information as possible.

In [4]:
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = text.strip(" ")
    text = re.sub(
        " +", " ", text
    ).strip()  # gets rid of multiple spaces and replace with a single
    return text

speeches["Text"] = speeches['Text'].map(lambda text: clean_text(text))

## 1.2 LOAD FAIRSEQ MODEL AND TRANSLATE

It takes just 2 lines of code to batch translate the speeches. The task was completed in about 7.5 minutes on my late-2015 iMac. Performance would vary according to your hardware/set-up.


In [5]:
en2ta = torch.hub.load('pytorch/fairseq', 'transformer.wmt20.en-ta')

Using cache found in /Users/empire/.cache/torch/hub/pytorch_fairseq_master


In [6]:
%%time
speeches["Translation"] = speeches["Text"].apply(
    lambda text: en2ta.translate(sent_tokenize(text))
)


CPU times: user 23min 25s, sys: 50 s, total: 24min 15s
Wall time: 7min 27s
Compiler : 330 ms


## 1.3 RE-ORG FOR CLARITY

In [7]:
cols1 = [
    "Date",
    "Speaker",
    "Title",
    "Text",
    "Translation",
    "URL",
]

speeches_translated = speeches[cols1]

In [8]:
speeches_translated.head()

,Date,Speaker,Title,Text,Translation,URL
0,2020-04-30,Lee Hsien Loong,May Day Message 2020,"This year, we mark May Day amidst difficult ci...","[இந்த வருடம், கடினமான சூழ்நிலைகளுக்கு இடையில் ...",https://www.pmo.gov.sg/Newsroom/PM-Lee-Hsien-L...
4,2020-07-27,Lee Hsien Loong,Cabinet Swearing-in Ceremony,Singaporeans have just gone through a crucial ...,[சிங்கப்பூரர்கள் மிகப்பெரிய நெருக்கடிக்கு இடைய...,https://www.pmo.gov.sg/Newsroom/Speech-by-PM-L...
7,2020-08-09,Lee Hsien Loong,National Day Message 2020,"Every year, rain or shine, Singaporeans come t...","[ஒவ்வொரு வருடமும், மழையோ, பிரகாசமோ, சிங்கப்பூர...",https://www.pmo.gov.sg/Newsroom/National-Day-M...


## 1.4 CHECK RESULTS

In [9]:
# OPTIONAL: output results for closer inspection

# speeches_translated.to_csv("../data/eng_to_tamil_speeches.csv", index=False)

In [10]:
# Check results of 1 speech translation
speeches_translated['Translation'].values[1]

['சிங்கப்பூரர்கள் மிகப்பெரிய நெருக்கடிக்கு இடையில் நடந்துமுடிந்த மிக முக்கியமான பொதுத்தேர்தலை இப்போதுதான் சந்தித்துள்ளனர்.',
 'COVID-19 மற்றும் வைரஸ் ஏற்படுத்திய பொருளாதார வீழ்ச்சி ஆகியவற்றைக் கையாண்டு ஆறு கடினமான மாதங்களைக் கடந்து வந்தோம்.',
 'இன்னும் வரவிருக்கும் கடினமான மாதங்கள் மற்றும் ஆண்டுகளைச் சமாளிக்க, அரசுக்கு ஒரு புதிய ஆணையையும், ஒரு முழு காலத்தையும் வழங்குவதற்காக இந்தத் தேர்தலுக்கு நான் அழைப்பு விடுத்தேன்.',
 'இதில் என்ன சிக்கல் இருக்கிறது என்பது சிங்கப்பூரர்களுக்குப் புரிந்தது.',
 'வேட்பாளர்களும், கட்சிகளும் என்ன செய்ய வேண்டும் என்பதை அவர்கள் கவனமாக பரிசீலித்தனர்.',
 'வாக்குப்பதிவு நாளன்று, வாக்காளர்கள் முழு பலத்துடன் வந்திருந்தனர்.',
 'அவர்கள் பொதுஜன பெரமுன கட்சிக்கு ஆட்சி அமைக்க ஒரு தெளிவான ஆணையை வழங்கினார்கள்.',
 'இந்தப் பெரும் பொறுப்பை சிங்கப்பூரர்கள் எங்களிடம் ஒப்படைத்திருப்பது குறித்து நானும் எனது குழுவினரும் மிகவும் தாழ்மையுடன் இருக்கிறோம்.',
 'எதிர்வரும் சவால்களைச் சமாளிக்கவும், சிங்கப்பூரைச் சிக்கலில் இருந்து மீட்டெடுக்கவும், உங்கள் சார்பாகச் செயல்படவும், விடாமுயற்

In [11]:
# English version for comparison

speeches_translated['Text'].values[1]

'Singaporeans have just gone through a crucial General Election, held in the midst of a tremendous crisis. We have been through six difficult months, dealing with COVID-19, and the economic downturn precipitated by the virus. I called this election to give the government a fresh mandate and a full term, to deal with the difficult months and years that still lie ahead. Singaporeans understood what was at stake. They considered carefully what the candidates and parties had to offer. And on Polling Day, voters turned out in full strength. They gave the PAP a clear mandate to form the government. My team and I are humbled that Singaporeans have entrusted us with this heavy responsibility. We will use our mandate to act on your behalf, vigorously and unremittingly, to deal with the challenges ahead, and lead Singapore out of the crisis. The work of our new Cabinet and government has already begun. The elections are behind us. Whatever our political persuasions, never forget that we are firs

# 2. TRANSLATE ENGLISH NEWS STORIES TO TAMIL

One likely use case of batch machine translation would be for newsrooms to convert their English-language stories to multiple languages without having to hire an army of translators. Let's try that here.

In [12]:
# this dataset is in the repo and was used in earlier machine translation exercises 
stories = pd.read_csv('../data/cna_sample.csv')

In [13]:
stories.head()

,Source,Date,Story_Headline,Story_Text,Word_Count,URL
0,CNA,2020-03-19,Wuhan's new COVID-19 cases could cease by mid-...,SHANGHAI: Wuhan is expected to see new coronav...,315,https://www.channelnewsasia.com/news/asia/coro...
1,CNA,2020-03-04,Researchers identify two novel coronavirus typ...,SHANGHAI: Scientists in China studying the vir...,481,https://www.channelnewsasia.com/news/asia/covi...
2,CNA,2020-03-05,Bethlehem's Church of the Nativity ordered clo...,BETHLEHEM: The Church of the Nativity was orde...,436,https://www.channelnewsasia.com/news/world/cor...
3,CNA,2020-03-19,NATAS travel fair in May cancelled as COVID-19...,SINGAPORE: Amid a COVID-19 outbreak that has p...,301,https://www.channelnewsasia.com/news/singapore...
4,CNA,2020-03-12,China's coronavirus epicentre Hubei sees singl...,BEIJING: China confirmed only eight new corona...,425,https://www.channelnewsasia.com/news/asia/coro...


## 2.1 CLEAN TEXT

Additional rules added to strip out promos and ads.

In [14]:
def clean_stories(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"Advertisement", " ", text)
    text = re.sub(
        r"Download our app or subscribe to our Telegram channel for the latest updates on the coronavirus outbreak: https://cna.asia/telegram",
        " ",
        text,
    )
    text = re.sub(
        r"Download our app or subscribe to our Telegram channel for the latest updates on the COVID-19 outbreak: https://cna.asia/telegram",
        " ",
        text,
    )
    text = text.strip(" ")
    text = re.sub(
        " +", " ", text
    ).strip()  # gets rid of multiple spaces and replace with a single
    return text

stories["Story_Text"] = stories['Story_Text'].map(lambda text: clean_stories(text))

## 2.2 TRANSLATE

In [15]:
%%time
stories['Headline_Translation'] = stories["Story_Headline"].apply(lambda text: en2ta.translate(text))

CPU times: user 2min 12s, sys: 1.07 s, total: 2min 13s
Wall time: 35.5 s


In [16]:
%%time
stories["Story_Translation"] = stories["Story_Text"].apply(
    lambda text: en2ta.translate(sent_tokenize(text))
)

CPU times: user 14min 11s, sys: 24.6 s, total: 14min 35s
Wall time: 3min 57s


## 2.3 RE-ORG FOR CLARITY

In [17]:
cols2 = [
    "Date",
    "Source",
    "Story_Headline",
    "Headline_Translation", 
    "Story_Text",
    "Story_Translation",
    "URL",
]

stories_translated = stories[cols2]

In [18]:
stories_translated.head()

,Date,Source,Story_Headline,Headline_Translation,Story_Text,Story_Translation,URL
0,2020-03-19,CNA,Wuhan's new COVID-19 cases could cease by mid-...,வுஹானின் புதிய COVID-19 வழக்குகள் மார்ச் மாதக்...,SHANGHAI: Wuhan is expected to see new coronav...,[ஷாங்காய்: மார்ச் மாதத்தின் மத்தியில் புதிய கொ...,https://www.channelnewsasia.com/news/asia/coro...
1,2020-03-04,CNA,Researchers identify two novel coronavirus typ...,சீனாவால் பாதிப்புகள் குறைந்து வருவதால் இரண்டு ...,SHANGHAI: Scientists in China studying the vir...,[ஷாங்காய்: புதிய கொரோனா வைரஸின் இரண்டு முக்கிய...,https://www.channelnewsasia.com/news/asia/covi...
2,2020-03-05,CNA,Bethlehem's Church of the Nativity ordered clo...,COVID-19 அச்சங்கள் குறித்து பெத்லகேமின் நேட்டி...,BETHLEHEM: The Church of the Nativity was orde...,[பெத்லெஹெம்: பாலஸ்தீன நகரான பெத்லெஹெம்மில் COV...,https://www.channelnewsasia.com/news/world/cor...
3,2020-03-19,CNA,NATAS travel fair in May cancelled as COVID-19...,COVID-19 தொற்று தொடர்ந்து வருவதால் மே மாதத்தில...,SINGAPORE: Amid a COVID-19 outbreak that has p...,[சிங்கபூர்: பல்வேறு நாடுகளையும் பயண ஆலோசனைகளை ...,https://www.channelnewsasia.com/news/singapore...
4,2020-03-12,CNA,China's coronavirus epicentre Hubei sees singl...,சீனாவின் கொரோனாவைரஸ் மையமான ஹூபே முதல் முறையாக...,BEIJING: China confirmed only eight new corona...,[பீஜிங்: ஹூபே மாகாணத்தில் எட்டு புதிய கொரோனாவை...,https://www.channelnewsasia.com/news/asia/coro...


## 2.4 CHECK RESULTS

In [19]:
# OPTIONAL: output results for closer inspection

#stories_translated.to_csv("../data/eng_to_tamil_stories.csv", index=False)

In [20]:
# check translated headlines
stories_translated['Headline_Translation'].values

array(['வுஹானின் புதிய COVID-19 வழக்குகள் மார்ச் மாதக் கடைசியில் நிறுத்தப்படலாம்: அறிக்கை',
       'சீனாவால் பாதிப்புகள் குறைந்து வருவதால் இரண்டு புதிரான கொரோனாவைரஸ் வகைகளை ஆராய்ச்சியாளர்கள் அடையாளம் காண்கின்றனர்',
       'COVID-19 அச்சங்கள் குறித்து பெத்லகேமின் நேட்டிவிட்டி தேவாலயம் மூட உத்தரவு',
       'COVID-19 தொற்று தொடர்ந்து வருவதால் மே மாதத்தில் நடாஸ் பயணக் கண்காட்சி ரத்து',
       'சீனாவின் கொரோனாவைரஸ் மையமான ஹூபே முதல் முறையாக ஒற்றை இலக்க வழக்குகளைக் காண்கிறது'],
      dtype=object)

In [21]:
# check 1 translated story

stories_translated['Story_Translation'].values[3]

['சிங்கபூர்: பல்வேறு நாடுகளையும் பயண ஆலோசனைகளை வழங்கத் தூண்டிய COVID-19 தொற்றுக்கு இடையில், சிங்கப்பூரின் தேசிய பயண முகவர்கள் சங்கம் (NATAS) மே மாதத்தில் நடக்கவிருந்த ஒரு கண்காட்சியை ரத்து செய்துள்ளது, மேலும் இந்த வருடத்தின் பிற்பகுதியில் மற்றொரு நிகழ்வுக்கு அதுபோல் செய்யலாமா என்பது குறித்து “உடனுக்குடன் முடிவெடுக்கும்”.',
 'இந்தச் சங்கம் பொதுவாக வருடத்திற்கு இரண்டு பிரம்மாண்டமான பயணக் கண்காட்சிகளை ஏற்பாடு செய்கிறது – ஒன்று பிப்ரவரி அல்லது மார்ச் மற்றும் இரண்டாவது ஆகஸ்ட் அல்லது செப்டம்பர்.',
 'இந்த வருடத்திற்கான தனது முதல் பயணக் கண்காட்சியை கடந்த மாதம் நடத்த அது திட்டமிட்டிருந்தது, ஆனால் COVID-19 பரவல் காரணமாக மே மாதத் துவக்கத்தில் அதை ஒத்திவைக்க அது முடிவு செய்தது.',
 'சிங்கப்பூர் மற்றும் பிற நாடுகளால் வழங்கப்பட்ட பயண ஆலோசனைகள், பொது சுகாதார அபாய மதிப்பீடு மற்றும் நிச்சயமற்ற பயண உணர்வு உள்ளிட்ட சமீபத்திய முன்னேற்றங்களைக் கவனமாகப் பரிசீலித்த பிறகு, தற்போது அந்த நிகழ்வை ரத்து செய்வதாக புதன்கிழமையன்று (மார் 18) நாடாஸ் வெளியிட்ட அறிக்கையில் கூறியது.',
 'இதில் பங்கேற்ற “பெரும்பான்மையான” வி

In [22]:
# English version for comparison
stories_translated['Story_Text'].values[3]

'SINGAPORE: Amid a COVID-19 outbreak that has prompted various countries to issue travel advisories, the National Association of Travel Agents Singapore (NATAS) has cancelled a fair scheduled for May and will “decide in due course” whether to do the same for another event later this year. The association typically organises two massive travel fairs a year - one in February or March and the second in August or September. It had planned to hold its first travel fair for the year last month but decided to postpone it to early-May due to the outbreak of COVID-19. In a statement on Wednesday (Mar 18), NATAS said it is now cancelling the event after carefully considering recent developments, which include travel advisories issued by Singapore and other countries, public health risk assessment and an uncertain travel sentiment. A “vast majority” of participating vendors were in favour of cancelling the event, it said, adding that it will offer a full rental refund to those vendors. NATAS is a